# Dataset

In [ ]:
from google.cloud import bigquery
import csv
import time
from tqdm.notebook import tqdm

In [13]:
%env GOOGLE_APPLICATION_CREDENTIALS=/home/rafaelkallis/.config/gcloud/application_default_credentials.json
client = bigquery.Client()

query = """
WITH
  -- label synonyms from Izadi et al.
  label_synonyms AS (
    SELECT 'bug' AS label, synonym FROM UNNEST(['bug', 'type: bug', 'kind/bug', 'crash', 'defect', 'type-defect', 'type:bug', 'browser bug', 'fix', 'fixed', 'bugfix', 'bug fix', 'resolution: fixed', 'troubleshooting', 'type/bug', 'bug report']) AS synonym
    UNION ALL
    SELECT 'feature', * FROM UNNEST(['feature', 'feature request', 'enhancement', 'improvement', 'type: feature', 'type:feature', 'new feature', 'kind/feature', 'kind/enhancement'])
    UNION ALL
    SELECT 'question', * FROM UNNEST(['question', 'faq', 'type: question', 'type:question'])
    UNION ALL
    SELECT 'documentation', * FROM UNNEST(['documentation', 'docs', 'doc', 'type: documentation', 'needs documentation', 'area/documentation', 'type: docs', 'type:docs', 'needs docs', 'wiki', 'kind/documentation', 'kind/docs'])
  ),
  close_events AS (
    SELECT payload
    FROM `githubarchive.day.2022*`
    WHERE
      _TABLE_SUFFIX BETWEEN '0101' AND '0102'
      -- _TABLE_SUFFIX BETWEEN '0101' AND '0631'
      AND type = 'IssuesEvent'
      AND JSON_EXTRACT_SCALAR(payload, '$.action') = 'closed'
      AND JSON_EXTRACT_SCALAR(payload, '$.issue.body') != 'null'
  ),
  nested_labels AS (
    SELECT
      ARRAY(
        SELECT LOWER(JSON_EXTRACT_SCALAR(label_payload, '$.name'))
        FROM UNNEST(JSON_EXTRACT_ARRAY(payload, '$.issue.labels')) AS label_payload
      ) AS labels,
      payload
    FROM close_events
  ),
  synonymized_labels AS (
    SELECT
      ARRAY(
        SELECT DISTINCT label_synonyms.label -- bug, feature, question, documentation
        FROM UNNEST(nested_labels.labels) AS label, label_synonyms
        WHERE label = label_synonyms.synonym
        ORDER BY label_synonyms.label
      ) AS labels,
      payload
    FROM nested_labels
  ),
  filtered_labels AS (
    SELECT labels, payload
    FROM synonymized_labels
    WHERE ARRAY_LENGTH(labels) = 1 -- exactly 1 label
  ),
  concatenated_labels AS (
    SELECT ARRAY_TO_STRING(labels, ',') AS labels, payload
    FROM filtered_labels
  )
SELECT
  JSON_EXTRACT_SCALAR(payload, '$.issue.id') AS id,
  labels,
  JSON_EXTRACT_SCALAR(payload, '$.issue.title') AS title,
  JSON_EXTRACT_SCALAR(payload, '$.issue.body') AS body,
  JSON_EXTRACT_SCALAR(payload, '$.issue.author_association') AS author_association
FROM concatenated_labels
"""

query_job = client.query(query)
rows = query_job.result()

with open(f"github-issue-classification.csv", "w", newline='') as f:
	writer = csv.DictWriter(
    f, 
    fieldnames=["id", "labels", "title", "body", "author_association"], 
    quoting=csv.QUOTE_NONE,
    escapechar='\\', 
  )
	writer.writeheader()
	for row in tqdm(rows, desc="BigQuery"):
		writer.writerow(row)


env: GOOGLE_APPLICATION_CREDENTIALS=/home/rafaelkallis/.config/gcloud/application_default_credentials.json


OSError: Project was not passed and could not be determined from the environment.

In [2]:
with open(f"github-issue-classification.csv", newline='') as f:
	n_lines = sum(1 for _ in csv.DictReader(f, quoting=csv.QUOTE_NONE, escapechar='\\'))
	print(n_lines)

0
